# MR Fingerprinting Dictionary Generation

In this example we demonstrate how to generate an MR Fingerprinting dictionary using a FISP type sequence with varying RF flip angles.

First, we install all the required dependencies as specified by the `Project.toml` and `Manifest.toml` files.

In [ ]:
using Pkg
Pkg.instantiate()

In [ ]:
# Load packages required for this notebook
using BlochSimulators
using ComputationalResources
using StructArrays

## Simulation setup

The 2D FISP sequence with time-varying RF excitations is defined in `src/sequences/fisp.jl`. 
We set all the required fields and create an instance of this type.

In [ ]:
nTR = 1000; # nr of TRs used in the simulation
RF_train = LinRange(1, 90, nTR) |> collect; # flip angle train
TR, TE, TI = 0.010, 0.005, 0.100; # repetition time, echo time, inversion delay
max_state = 64; # maximum number of configuration states to keep track of

sequence = FISP2D(complex.(RF_train), TR, TE, max_state, TI);

Next, set the desired input tissue properties for which the
FISP sequence response will be simulated

In [ ]:
T₁_range = range(start=0.1, stop=5.0, length=50); # T₁ range
T₂_range = range(start=0.025, stop=0.5, length=50); # T₂ range

# Generate valid combinations of T₁ and T₂ and store them in custom T₁T₂ struct
parameters = ([T₁T₂(T₁,T₂) for T₁ ∈ T₁_range, T₂ ∈ T₂_range if T₁ > T₂]);

println("Length parameters: $(length(parameters))")

Now we can perform the simulations using different hardware resources

## Single-threaded CPU

Note that the first time a function is called in a Julia session,
a precompilation procedure starts and the runtime for subsequent function
calls are significantly faster

In [ ]:
@time dictionary = simulate_magnetization(CPU1(), sequence, parameters)

The second time a function is called with arguments of similar types,
the pre-compiled version is called immediatly.

In [ ]:
@time dictionary = simulate_magnetization(CPU1(), sequence, parameters);

Note that the dictionary is a matrix with the magnetization response (at echo times)
for all combinations of input tissue properties

In [ ]:
@assert size(dictionary) == (nTR, length(parameters))

## Multi-threaded CPU

To use multiple threads, Julia must be started with the `--threads=auto`
flag (or some integer instead of `auto`). Alternatively, set the
environent variable `JULIA_NUM_THREADS` to the desired number of threads
in your shell before starting Julia.

In [ ]:
# Check the number of available threads
println("Current number of threads: $(Threads.nthreads())")

We can simulate in a multi-threaded fashion with the following syntax:

In [ ]:
@time dictionary = simulate_magnetization(CPUThreads(), sequence, parameters);

In fact, BlochSimulators defaults to using CPUThreads() so we can also call

In [ ]:
@time dictionary = simulate_magnetization(sequence, parameters);

## Distributed CPU

For distributed CPU mode, use the Distribute packages (ships with Julia)
to add workers first

In [ ]:
using Distributed
addprocs(4, exeflags="--project=.")


@everywhere using BlochSimulators

println("Current number of workers: $(nworkers())")
@time dictionary = simulate_magnetization(CPUProcesses(), sequence, parameters);

Alternatively, if you can ssh into some other machine,
you can add CPUs from that machine as follows:

`addprocs([("12.345.67.89", 4)], exeflags="--project=."`

Or, if you want to run this code on cluster with a queuing system, use ClusterManagers package.

After workers have been added, load BlochSimulators on all workers
and then start a distributed dictionary generation with:

## GPU (CUDA device)

First, let's check if a CUDA device is available

In [ ]:
using CUDA

@assert CUDA.has_cuda_gpu()

println("Active CUDA device:");
CUDA.device()

To perform simulations on GPU, we first convert the sequence and parameters
to single precision and then send them to the gpu. 

To this end, BlochSimulators
exports a `f32` function which recursively converts inputs to single precision.


Similarly, a `gpu` function is exported which sends the input to the GPU.

In [ ]:
cu_sequence = sequence |> f32 |> gpu;
cu_parameters = parameters |> f32 |> gpu;

Remember, the first time a compilation procedure takes place which, especially
on GPU, can take some time.

In [ ]:
CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);

In [ ]:
CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);

Now let's increase the number of tissue property combinations for which
simulations are performed:

In [ ]:
T₁ = rand(500_000)
T₂ = 0.1 * T₁
cu_parameters = (@parameters T₁ T₂) |> f32 |> gpu

CUDA.@time dictionary = simulate_magnetization(CUDALibs(), cu_sequence, cu_parameters);